In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
@import url('http://fonts.googleapis.com/css?family=Crimson+Text');
@import url('http://fonts.googleapis.com/css?family=Source+Code+Pro');

/* Change code font */
.CodeMirror pre {
    font-family: 'Input Mono Narrow', 'Source Code Pro', Consolas, monocco, monospace;
}

div.input_area {
    border-color: rgba(0,0,0,0.10);
}

div.text_cell {
    max-width: 105ex; /* instead of 100%, */
}

div.text_cell_render {
    font-family: "Crimson Text";
    font-size: 12pt;
    line-height: 145%; /* added for some line spacing of text. */
}

div.text_cell_render h1,
div.text_cell_render h2,
div.text_cell_render h3,
div.text_cell_render h4,
div.text_cell_render h5,
div.text_cell_render h6 {
    font-family: 'Crimson Text';
}

.rendered_html pre,
.rendered_html code {
    font-size: medium;
}

.rendered_html ol {
    list-style:decimal;
    margin: 1em 2em;
}

.prompt.input_prompt {
    color: rgba(0,0,0,0.5);
}

.cell.command_mode.selected {
    border-color: rgba(0,0,0,0.1);
}

.cell.edit_mode.selected {
    border-color: rgba(0,0,0,0.15);
    box-shadow: 0px 0px 5px #f0f0f0;
    -webkit-box-shadow: 0px 0px 5px #f0f0f0;
}

div.output_scroll {
    -webkit-box-shadow: inset 0 2px 8px rgba(0,0,0,0.1);
    box-shadow: inset 0 2px 8px rgba(0,0,0,0.1);
    border-radious: 2px;
}

#menubar .navbar-inner {
    -webkit-box-shadow: none;
    box-shadow: none;
    border-radius: 0;
    border: none;
    font-family: lato;
    font-weight: 400;
}

.navbar-fixed-top .navbar-inner,
.navbar-static-top .navbar-inner {
    box-shadow: none;
    -webkit-box-shadow: none;
    border: none;
}

div#notebook_panel {
    box-shadow: none;
    -webkit-box-shadow: none;
    border-top: none;
}

div#notebook {
    border-top: 1px solid rgba(0,0,0,0.15);
}

/* 
    This is a lazy fix, we *should* fix the 
    background for each Bootstrap button type
*/
#site * .btn {
    -webkit-box-shadow: none;
    box-shadow: none;
}


span.ansiblack {color: #073642;}
span.ansiblue {color: #2aa198;}
span.ansigray {color: #839496;}
span.ansigreen {color: #859900;}
span.ansipurple {color: #6c71c4;}
span.ansired {color: #dc322f;}
span.ansiyellow {color: #b58900;}

div.output_stderr {background-color: #dc322f;}
div.output_stderr pre {color: #eee8d5;}

.cm-s-ipython.CodeMirror {background: #fdf6e3; color: #073642;}
.cm-s-ipython div.CodeMirror-selected {background: #eee8d5 !important;}
.cm-s-ipython .CodeMirror-gutters {background: #fdf6e3; border-right: 0px;}
.cm-s-ipython .CodeMirror-linenumber {color: #839496;}
.cm-s-ipython .CodeMirror-cursor {border-left: 1px solid #657b83 !important;}

.cm-s-ipython span.cm-comment {color: #d33682;}
.cm-s-ipython span.cm-atom {color: #6c71c4;}
.cm-s-ipython span.cm-number {color: #6c71c4;}

.cm-s-ipython span.cm-property, .cm-s-ipython span.cm-attribute {color: #859900;}
.cm-s-ipython span.cm-keyword {color: #dc322f;}
.cm-s-ipython span.cm-string {color: #b58900;}
.cm-s-ipython span.cm-operator {color: #d33682;}
.cm-s-ipython span.cm-builtin {color: #6c71c4;}

.cm-s-ipython span.cm-variable {color: #859900;}
.cm-s-ipython span.cm-variable-2 {color: #268bd2;}
.cm-s-ipython span.cm-def {color: #cb4b16;}
.cm-s-ipython span.cm-error {background: #dc322f; color: #657b83;}
.cm-s-ipython span.cm-bracket {color: #586e75;}
.cm-s-ipython span.cm-tag {color: #dc322f;}
.cm-s-ipython span.cm-link {color: #6c71c4;}

.cm-s-ipython .CodeMirror-matchingbracket { text-decoration: underline; color: #073642 !important;}
th, td { 
      font-size: 18px;
      border-collapse: collapse;
      border-width:3px;
    }
    </style>

""")

# Deriving all vulnerability KPIs for cities in the US

## This notebook uses data generated from the different vulnerability_* notebooks. 

Description of KPIs: (the climate hazard-specific ones can be found in the different notebooks)

| KPI Name             | KPI Interpretation                                            | Public Data Source       |
|----------------------|---------------------------------------------------------------|--------------------------|
| human_pressure       | Human pressure on environment footprint                       | SEDAC                    |
| aqi_mean             | Mean of air quality index over last three years               | Kaggle Dataset           |
| daqi                 | Change in air quality index year-by-year                      | Kaggle Dataset           |
| air_pollution_trends | Trend in air pollution (PM2.5)                                | County Health Rankings   |
| pop_density          | population density                                            | Survey                   |
| emissions_per_capita | emissions per capita in different counties                    | Jones & Kammen (2014)    |
| emissions_per_km2    | emissions per land area in different counties                 | Jones & Kammen (2014)    |
| expected_annual_loss | overall expected annual loss from different climate disasters | FEMA National Risk Index |
| risk_city_sentiment  | city sentiment on the risks they face                         | CDP Survey               |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.preprocessing import RobustScaler, QuantileTransformer
import rasterio
from matplotlib import colors
from scipy import stats

from tqdm import *
from pyproj import Transformer
import ast
from sklearn.preprocessing import RobustScaler, QuantileTransformer, StandardScaler


In [ ]:
path = "../input/externald/Supplementary Data/" 

In [ ]:
vulnerability = pd.read_csv(path + "cities_updated_geo_us_2020.csv")
acct_number = vulnerability["Account Number"]
counties = (vulnerability['counties'])
counties_weights = (vulnerability['counties_weights'])
states = (vulnerability['state'])
flag_multi_counties = vulnerability['flag_multiple_counties']
match_fips = vulnerability['FIPS']
MSA = (vulnerability['MSA'])
match_fips = [ast.literal_eval(match_fips[i]) if flag_multi_counties[i] else int(match_fips[i]) for i in range(len(match_fips))]
cities = vulnerability["city_bing"]

In [ ]:
def standardize_rank(arr, direction=1):
    rank = (stats.mstats.rankdata(np.ma.masked_invalid(arr)))
    rank[rank == 0] = np.nan
    if(direction == 1):
        return rank/(np.nanmax(rank)) 
    else:
        return 1-(rank/(np.nanmax(rank)))


## First,  we use SEDAC data to calculate the human footprint on the environment

In [ ]:
fp = path + 'Human_Footprint/human_footprint.tif'
img = rasterio.open(fp)
img_array = img.read(1)
transformer = Transformer.from_crs("epsg:4326", img.crs)

def get_value(raster, array, lon, lat):
    coords = transformer.transform(lat, lon) 
    py, px = raster.index(coords[0], coords[1])
    return array[py, px]

plt.imshow(img_array, norm=colors.LogNorm())

In [ ]:
human_pressure = []
for i in tqdm(range(len(vulnerability))):
    coords = ((vulnerability['loc'].iloc[i].replace("POINT (", "")).replace(")","")).split()
    lon = float(coords[0])
    lat = float(coords[1])
    human_pressure.append(get_value(img, img_array, lon, lat))
    
direction_human_pressure = 1.0
human_pressure_score = standardize_rank(np.array(human_pressure), direction=direction_human_pressure)

Let's calculate some emission / air pollution statistics. 

In [ ]:
aqi = pd.read_csv(path + 'AQI_by_year.csv')
accts_2020 = list(vulnerability["Account Number"])

aqi_mean = []
for i in range(len(accts_2020)):
    try:
        aa = (aqi[aqi["account_number"] == accts_2020[i]]).sort_values("year")["arithmetic_mean"]
        da = np.mean(aa.iloc[-3:])
        aqi_mean.append(da)
    except:
        aqi_mean.append(np.nan)
    
daqi = []
for i in range(len(accts_2020)):
    try:
        aa = (aqi[aqi["account_number"] == accts_2020[i]]).sort_values("year")["arithmetic_mean"]
        da = np.mean(np.diff(aa)/aa.iloc[0])
        daqi.append(da)
    except:
        daqi.append(np.nan)


In [ ]:
direction_air_quality = 1.0
aqi_score = standardize_rank(np.array(aqi_mean), direction=direction_air_quality)

direction_air_quality_trend = 1.0
daqi_score = standardize_rank(np.array(daqi), direction=direction_air_quality_trend)

In [ ]:
trends = pd.read_csv(path + "CHR_trends_csv_2020.csv", encoding='latin-1')
ct = np.char.zfill(np.array(trends['countycode']).astype(str), 3)
st = np.char.zfill(np.array(trends['statecode']).astype(str), 2)
trends['fips'] = np.core.defchararray.add(st, ct).astype(int)

In [ ]:
air_pollution_trends = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = match_fips[i]
        cdf = trends[(trends['measurename'] == "Air pollution - particulate matter") & (trends['fips'] == fips)]
        rv = np.array(cdf['rawvalue']).astype(float)
        rvd = (np.mean(rv[-3:]) - np.mean(rv[0:3]))/(np.mean(rv[0:3]))
        air_pollution_trends.append(np.float(rvd))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            cdf = trends[(trends['measurename'] == "Air pollution - particulate matter") & (trends['fips'] == fips[j])]
            rv = np.array(cdf['rawvalue']).astype(float)
            rvd = (np.mean(rv[-3:]) - np.mean(rv[0:3]))/(np.mean(rv[0:3]))
            temp.append(np.float(rvd))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        air_pollution_trends.append(weighted_avg)

direction_air_pollution_trend = 1.0
air_pollution_trend_score = standardize_rank(np.array(air_pollution_trends), direction=direction_air_pollution_trend)

In [ ]:
cities_2020 = pd.read_csv("/kaggle/input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2020_Full_Cities_Dataset.csv")

cities_disc_2020 = pd.read_csv("/kaggle/input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv")

cities_2020_merged = pd.merge(cities_2020, cities_disc_2020, on="Account Number", how="outer", copy=False)
unique_20 = np.unique(cities_disc_2020[cities_disc_2020["Country"] == "United States of America"]["Account Number"])
cities_2020_merged_sub = cities_2020_merged[np.in1d(cities_2020_merged["Account Number"], unique_20)]

def fetch_answer_individual(account, question_number, org_type="city", year=2018, corp_res_type=None, column_number=None, row_number=None):
    if(org_type=="city"):
        df = cities_2020_merged_sub
        subset = df[df['Question Number'] == question_number]
        answer = subset[subset["Account Number"] == account]
        if((column_number is not None) and (row_number is None)):
            answer = answer[answer["Column Number"] == column_number]
        elif((column_number is not None) and (row_number is not None)):
            answer = answer[(answer["Column Number"] == column_number) & (answer["Row Number"] == row_number)]
        elif((column_number is None) and (row_number is not None)):
            answer = answer[answer["Row Number"] == row_number]
        else:
            pass
    elif(org_type=="corp" and corp_res_type=="cc"):
        df = corporations_2020
        subset = df[df['question_number'] == question_number]
        answer = subset[subset["account_number"] == account]
        if((column_number is not None) and (row_number is None)):
            answer = answer[answer["column_number"] == column_number]
        elif((column_number is not None) and (row_number is not None)):
            answer = answer[(answer["column_number"] == column_number) & (answer["row_number"] == row_number)]
        elif((column_number is None) and (row_number is not None)):
            answer = answer[answer["row_number"] == row_number]
        else:
            pass
    elif(org_type=="corp" and corp_res_type=="ws"):
        df = all_corps_cc[year]
        subset = df[df['question_number'] == question_number]
        answer = subset[subset["account_number"] == account]
    else:
        print("Something went wrong. Try again.")
    
    return answer

In [ ]:
population = np.array([float(fetch_answer_individual(accts_2020[i], '0.5', column_number=1.0)["Response Answer"]) for i in range(len(accts_2020))])
areas_km2 = np.array([float(fetch_answer_individual(accts_2020[i], '0.6')["Response Answer"]) for i in range(len(accts_2020))])
areas_km2[cities == "Greenbelt, MD"] /= 10**3 # Greenbelt reported in the wrong units
pop_density = population/(areas_km2)
direction_pop_density = 1
log_pop_density_score = standardize_rank(np.log10(pop_density), direction=direction_pop_density)

In [ ]:
carbon_footprint = pd.read_excel("/kaggle/input/joneskammen/Jones-Kammen-2014-Zip-City-County-Results.xlsx", sheet_name="County results")
carbon_footprint["County"] = carbon_footprint["County"].str.title()

In [ ]:
emissions_per_capita = []
for i in range(len(counties)):
    if(not flag_multi_counties[i]):
        ct = ((counties[i].replace(" County", "")).replace("'", "").replace(".","").replace(" Parish", "")).title()
        cdf = carbon_footprint[(carbon_footprint["County"].str[:-1] == ct) & (carbon_footprint["State"] == states[i])]
        emissions_per_capita.append(float(cdf["Total County Carbon Footprint (tCO2e/yr)"]/cdf["Population"]))
    else:
        fips = ast.literal_eval(counties[i])
        temp = []
        for j in range(len(fips)):
            ct = ((fips[j].replace(" County", "")).replace("'", "").replace(".","").replace(" Parish", "")).title()
            cdf = carbon_footprint[(carbon_footprint["County"].str[:-1] == ct) & (carbon_footprint["State"] == states[i])]
            temp.append((float(cdf["Total County Carbon Footprint (tCO2e/yr)"]/cdf["Population"])))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        emissions_per_capita.append(weighted_avg)
        

In [ ]:
emissions_per_km2 = []
for i in range(len(counties)):
    if(not flag_multi_counties[i]):
        ct = ((counties[i].replace(" County", "")).replace("'", "").replace(".","").replace(" Parish", "")).title()
        cdf = carbon_footprint[(carbon_footprint["County"].str[:-1] == ct) & (carbon_footprint["State"] == states[i])]
        emissions_per_km2.append(float(cdf["Total County Carbon Footprint (tCO2e/yr)"]/areas_km2[i]))
    else:
        fips = ast.literal_eval(counties[i])
        temp = []
        for j in range(len(fips)):
            ct = ((fips[j].replace(" County", "")).replace("'", "").replace(".","").replace(" Parish", "")).title()
            cdf = carbon_footprint[(carbon_footprint["County"].str[:-1] == ct) & (carbon_footprint["State"] == states[i])]
            temp.append((float(cdf["Total County Carbon Footprint (tCO2e/yr)"]/areas_km2[i])))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        emissions_per_km2.append(weighted_avg)
        

In [ ]:
direction_emissions_per_capita = 1
emissions_per_capita_score = standardize_rank(np.array(emissions_per_capita), direction=direction_emissions_per_capita)

direction_emissions_per_km2 = 1
emissions_per_km2_score = standardize_rank(np.log10(emissions_per_km2), direction=direction_emissions_per_km2)

In [ ]:
nri = pd.read_csv(path + "NRI_Table_Counties.csv")

def nri_select(field):
    res = []
    for i in range(len(match_fips)):
        if(not flag_multi_counties[i]):
            fips = int(match_fips[i])
            cdf = nri[nri["STCOFIPS"] == fips]
            try:
                res.append(float(cdf[field]))
            except:
                res.append(np.nan)
        else:
            fips = match_fips[i]
            temp = []
            for j in range(len(fips)):
                cdf = nri[nri["STCOFIPS"] == fips[j]]
                try:
                    temp.append(float(cdf[field]))
                except:
                    temp.append(np.nan)
            ar = np.array(temp)
            weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
            res.append(weighted_avg)
    return np.array(res)

In [ ]:
expected_annual_loss = nri_select("EAL_SCORE")
direction_expected_annual_loss = 1
expected_annual_loss_score = standardize_rank(expected_annual_loss, direction=direction_expected_annual_loss)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

sid = SentimentIntensityAnalyzer()

def polarity_score_city(question, column, sent_thresh=2):
    subset = cities_2020[cities_2020['Question Number'] == question]
    subset_city_answers = subset[subset["Column Number"] == column]
    
    compound_score = []
    for i in range(len(acct_number)):
        ss = list(subset_city_answers[subset_city_answers["Account Number"] == acct_number[i]]["Response Answer"])
        ss = ' '.join(map(str, ss))
        sents = tokenize.sent_tokenize(ss)
        if(len(sents) > sent_thresh):
            cc = 0
            for s in sents:
                r = sid.polarity_scores(s)
                cc += r['compound']
            avg_cc = cc/len(sents)
            compound_score.append(avg_cc)
        else:
            compound_score.append(np.nan)
    return (compound_score)

In [ ]:
risk_city_sentiment = polarity_score_city("2.1", 12.0)
risk_city_sentiment_ranked = standardize_rank(risk_city_sentiment, -1)

In [ ]:
vulnerability_raw = vulnerability.copy()
vulnerability_ranked = vulnerability.copy()
vulnerability_aggr = vulnerability.copy()

In [ ]:
### EXPOSURE

vulnerability_raw["log_pop_density"] = np.log10(pop_density)
vulnerability_raw["human_pressure"] = human_pressure
vulnerability_raw["expected_annual_loss"] = expected_annual_loss
vulnerability_raw["risk_city_sentiment"] = risk_city_sentiment

### EMISSIONS & AIR POLLUTION

vulnerability_raw["aqi_mean"] = aqi_mean
vulnerability_raw["daqi"] = daqi
vulnerability_raw["air_pollution_trend"] = air_pollution_trends
vulnerability_raw["emissions_per_capita"] = emissions_per_capita
vulnerability_raw["emissions_per_km2"] = emissions_per_km2

vulnerability_raw.to_csv("vulnerability_cities_us_general_kpis_raw.csv")

In [ ]:
### EXPOSURE

vulnerability_ranked["log_pop_density_score"] = log_pop_density_score
vulnerability_ranked["human_pressure_score"] = human_pressure_score
vulnerability_ranked["expected_annual_loss_score"] = expected_annual_loss_score
vulnerability_ranked["risk_city_sentiment_score"] = risk_city_sentiment_ranked

### EMISSIONS & AIR POLLUTION

vulnerability_ranked["aqi_score"] = aqi_score
vulnerability_ranked["daqi_score"] = daqi_score
vulnerability_ranked["air_pollution_trend_score"] = air_pollution_trend_score
vulnerability_ranked["emissions_per_capita_score"] = emissions_per_capita_score
vulnerability_ranked["emissions_per_km2_score"] = emissions_per_km2_score
vulnerability_ranked.to_csv("vulnerability_cities_us_general_kpis_ranked.csv")

In [ ]:
def aggr_and_rescale(arr):
    avg_arr = np.nanmean(arr, axis=1)
    rescaled_avg_arr = (avg_arr - np.nanmin(avg_arr))/(np.nanmax(avg_arr) - np.nanmin(avg_arr))
    return rescaled_avg_arr

In [ ]:
all_vul_general_kpis = np.c_[log_pop_density_score, human_pressure_score, expected_annual_loss_score, 
                         risk_city_sentiment_ranked, aqi_score, daqi_score, air_pollution_trend_score, emissions_per_capita_score,
                       emissions_per_km2_score]

aggr_vul_general_kpis = aggr_and_rescale(all_vul_general_kpis)

vulnerability_aggr["aggr_vul_general_kpis"] = aggr_vul_general_kpis
vulnerability_aggr.to_csv("vulnerability_cities_us_general_aggr_kpis.csv")

In [ ]:
vul_aggr_heat = pd.read_csv("../input/derivedkpis/KPIs/vulnerability_cities_us_heat_aggr_kpis.csv")
vul_aggr_cold = pd.read_csv("../input/derivedkpis/KPIs/vulnerability_cities_us_cold_aggr_kpis.csv")
vul_aggr_drought = pd.read_csv("../input/derivedkpis/KPIs/vulnerability_cities_us_drought_aggr_kpis.csv")
vul_aggr_flood = pd.read_csv("../input/derivedkpis/KPIs/vulnerability_cities_us_floods_aggr_kpis.csv")
vul_aggr_severe = pd.read_csv("../input/derivedkpis/KPIs/vulnerability_cities_us_severe_aggr_kpis.csv")

In [ ]:
aggr_vul_general_kpis = aggr_and_rescale(all_vul_general_kpis)
aggr_vul_all = np.c_[aggr_vul_general_kpis, vul_aggr_heat["aggr_heat_kpis"], vul_aggr_cold["aggr_cold_kpis"], vul_aggr_drought["aggr_drought_kpis"]
                    , vul_aggr_flood["aggr_floods_kpis"], vul_aggr_severe["aggr_severe_kpis"]]

aggr_vul_all_kpis = aggr_and_rescale(aggr_vul_all)

vulnerability_aggr_all = vulnerability.copy()

vulnerability_aggr_all["aggr_heat_kpis"] = vul_aggr_heat["aggr_heat_kpis"]
vulnerability_aggr_all["aggr_cold_kpis"] = vul_aggr_cold["aggr_cold_kpis"]
vulnerability_aggr_all["aggr_drought_kpis"] = vul_aggr_drought["aggr_drought_kpis"]
vulnerability_aggr_all["aggr_flood_kpis"] = vul_aggr_flood["aggr_floods_kpis"]
vulnerability_aggr_all["aggr_severe_kpis"] = vul_aggr_severe["aggr_severe_kpis"]
vulnerability_aggr_all["aggr_vul_all_kpis"] = aggr_vul_all_kpis

vulnerability_aggr_all.to_csv("vulnerability_cities_us_all_aggr_kpis.csv")